In [1]:
import pandas as pd

In [29]:
df = pd.read_csv('/local0/batch_task.csv', names=['task_name', 'instance_num', 'job_name', 'task_type', 'status' , 'start_time'
, 'end_time', 'plan_cpu', 'plan_mem'])
print(df.columns)

# Filter jobs with dependency, dataset has two types of batch jobs, w/wo DAG info and 
df = df.loc[~df['task_name'].str.startswith('task_', na=False)]

Index(['task_name', 'instance_num', 'job_name', 'task_type', 'status',
       'start_time', 'end_time', 'plan_cpu', 'plan_mem'],
      dtype='object')


In [30]:
jobs_grp = df.groupby('job_name')
print('# of submitted DAGs', jobs_grp.ngroups)

# of submitted DAGs 3370756


In [31]:
task_pdag = jobs_grp['task_name'].agg(['count'])
print(task_pdag.idxmax())

count    j_3989171
dtype: object


In [35]:
tasks_i = jobs_grp.get_group('j_3989171').reset_index()['task_name'].values
print(tasks_i)

['M49' 'M127' 'R142_141' 'R181_180' 'M202' 'M92' 'R16_3' 'R63_62' 'R97_96'
 'R87_86' 'M151' 'R126_119' 'R13_12' 'R150_149' 'R125_117_119' 'R36_25'
 'R187_186' 'R81_80' 'M98' 'M135' 'M190' 'R160_159' 'R166_165' 'M23'
 'R140_139' 'M66' 'R110_109' 'R164_163' 'R191_190' 'R168_167' 'M67'
 'R124_123' 'M56' 'M121' 'M180' 'M145' 'M178' 'M8' 'R201_200' 'R77_76'
 'R185_184' 'R148_147' 'M80' 'M169' 'M163' 'M85' 'M141' 'R114_113'
 'R134_133' 'R101_100' 'M74' 'R183_182' 'J68_66_67' 'R21_17' 'M176'
 'R42_10' 'R31_10' 'R179_178' 'M171' 'R43_14' 'R118_117' 'M10' 'R162_161'
 'R203_202' 'R40_17' 'J29_2_28' 'R158_157' 'R79_78' 'R26_25' 'R156_155'
 'M157' 'M165' 'R5_1_4' 'R116_115' 'R19_17' 'R33_17' 'R59_58' 'R69_68'
 'J84_66' 'R45_17' 'M88' 'R193_192' 'M90' 'M62' 'R130_129' 'M12' 'M102'
 'M192' 'M100' 'R46_17' 'R112_111' 'R89_88' 'R61_51' 'M153' 'R44_25'
 'R132_131' 'M149' 'M94' 'R144_143' 'M188' 'R24_23' 'R83_82' 'M133'
 'R65_62' 'R108_107' 'R7_6' 'M76' 'J174_2_173' 'J113_66' 'J86_66_84_85'
 'R122_121' 

In [49]:
import re

adjls = {}
for t in tasks_i:
    deps = t.split('_')
    for x in deps:
        nid = re.sub("\D", "", x)
        nodes[nid] = []

for t in tasks_i:
    deps = t.split('_')
    for x in deps:
        nid = re.sub("\D", "", x)
        nodes[nid] = []

for t in tasks_i:
    chn = deps = t.split('_')[0]
    deps = t.split('_')[1:]
    for x in deps:
        nid = re.sub("\D", "", x)
        nodes[nid].append(re.sub("\D", "", chn))
        
print(nodes)

{'49': ['50'], '127': ['128'], '142': [], '141': ['142'], '181': [], '180': ['181'], '202': ['203'], '92': ['93'], '16': ['17'], '3': ['16', '4', '173'], '63': [], '62': ['63', '65', '64'], '97': [], '96': ['97'], '87': [], '86': ['87'], '151': ['152'], '126': [], '119': ['126', '125', '120'], '13': ['14'], '12': ['13'], '150': [], '149': ['150'], '125': [], '117': ['125', '118'], '36': [], '25': ['36', '26', '44', '41'], '187': [], '186': ['187'], '81': [], '80': ['81'], '98': ['99'], '135': ['137'], '190': ['191'], '160': [], '159': ['160'], '166': [], '165': ['166'], '23': ['24'], '140': [], '139': ['140'], '66': ['68', '84', '113', '86', '115', '93', '117', '119', '95', '137', '146', '71', '138'], '110': [], '109': ['110'], '164': [], '163': ['164'], '191': [], '168': [], '167': ['168'], '67': ['68'], '124': [], '123': ['124'], '56': ['57', '60'], '121': ['122'], '145': ['146'], '178': ['179'], '8': ['9'], '201': [], '200': ['201'], '77': [], '76': ['77'], '185': [], '184': ['185']